<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/Econophysics/NR7%3FStocksYahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta

# Define the stocks to be analyzed
stocks = ['AAPL', 'GOOG', 'AMZN', 'MSFT']

# Define the start and end date of the historical data
start_date = '2010-01-01'
end_date = '2023-04-16'

# Retrieve historical price data for the stocks from Yahoo Finance
prices = pd.DataFrame()
for stock in stocks:
    try:
        data = yf.download(stock, start=start_date, end=end_date)
        if 'Close' in data.columns:
            prices[stock] = data['Close']
        else:
            print(f"Ooops!!! It's True, the series 'Close' was not found in {stock}")
    except Exception as e:
        print(f"Error while fetching data for {stock}: {e}")

# ... (The rest of the code remains the same)

# Compute the high-low range for each day over the last seven days
highs = prices.rolling(7).max()
lows = prices.rolling(7).min()
ranges = highs - lows
nr7 = ranges.iloc[-1].sort_values()

# Calculate additional technical indicators using pandas_ta
sma50 = prices.ta.sma(length=50)
sma200 = prices.ta.sma(length=200)
rsi = prices.ta.rsi(length=14)
try:
    macd = prices.ta.macd(fast=12, slow=26, signal=9)
    macdh = macd['MACDh']
except KeyError:
    macdh = pd.Series(np.nan, index=prices.index)
    print("Unable to calculate MACD")

# Perform fundamental analysis using yfinance
fundamentals = pd.DataFrame()
for stock in stocks:
    info = yf.Ticker(stock).info
    eps_growth = info.get('forwardEpsGrowth')
    pe_ratio = info.get('trailingPE')
    roe = info.get('returnOnEquity')
    fundamentals.loc['EPS Growth (next year)', stock] = eps_growth
    fundamentals.loc['PE Ratio', stock] = pe_ratio
    fundamentals.loc['Return on Equity (TTM)', stock] = roe

# Combine technical and fundamental analyses to generate a final list of potential trades
trades = pd.DataFrame(index=nr7.index)
trades['Close'] = prices.iloc[-1]
trades['50-day SMA'] = sma50.iloc[-1]
trades['200-day SMA'] = sma200.iloc[-1]
trades['RSI'] = rsi.iloc[-1]
trades['MACD'] = macdh.iloc[-1]
trades = trades.join(fundamentals)
trades = trades.dropna()

# Display the potential trades
print(trades)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[X] Ooops!!! It's True, the series 'close' was not found in AAPL, GOOG, AMZN, MSFT
[X] Ooops!!! It's True, the series 'close' was not found in AAPL, GOOG, AMZN, MSFT
[X] Ooops!!! It's True, the series 'close' was not found in AAPL, GOOG, AMZN, MSFT
[X] Ooops!!! It's True, the series 'close' was not found in AAPL, GOOG, AMZN, MSFT
Unable to calculate MACD
Empty DataFrame
Columns: [Close, 50-day SMA, 200-day SMA, RSI, MACD, AAPL, GOOG, AMZN, MSFT]
Index: []
